In [1]:
import ROOT
from OutFunc import OutFunc

In [2]:
ROOT.TMVA.Tools.Instance()

In [3]:
#Set up an output file and book some histograms
#histFile = ROOT.TFile("analysis.root", "RECREATE")
hSigTrain = ROOT.TH1D("hSigTrain", "BDT, Signal (Train)", 100, -1.0, 1.0)
hBkgTrain = ROOT.TH1D("hBkgTrain", "BDT, Background (Train)", 100, -1.0, 1.0)
hSigTest  = ROOT.TH1D("hSigTest",  "BDT, Signal (Test)", 100, -1.0, 1.0)
hBkgTest  = ROOT.TH1D("hBkgTest",  "BDT, Background (Test)", 100, -1.0, 1.0)

In [4]:
# Set up the OutFunc object.  First argument must be one of the classifiers.
# 4th argument is offset for contour.
# 5th argument is bool array indicating which variables were used in training
dir = "dataset/weights/"
prefix = "tmvaTest"
name = "BDTcheck"
tCut = 0.0
useVar = [False, True, True, True, True, True, True]

testStat = OutFunc(name, dir, prefix, tCut, useVar)

                         : Booking "BDTcheck" of type "BDT" from dataset/weights//tmvaTest_BDTcheck.weights.xml.
                         : Reading weight file: dataset/weights//tmvaTest_BDTcheck.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
                         : Booked classifier "BDTcheck" of type: "BDT"


In [5]:
# === Function to fill histograms ===
def fill_histograms(input_path, is_test):
    inputFile = ROOT.TFile(input_path)
    sig = inputFile.Get("sig")
    bkg = inputFile.Get("bkg")

    for i, tree in enumerate([sig, bkg]):
        numEntries = tree.GetEntries()
        print(f"{'Test' if is_test else 'Train'} Tree: {'sig' if i==0 else 'bkg'}, Entries: {numEntries}")

        for j in range(numEntries):
            tree.GetEntry(j)
            t = testStat.val(tree)

            if i == 0:  # signal
                if is_test:
                    hSigTest.Fill(t)
                else:
                    hSigTrain.Fill(t)
            else:       # background
                if is_test:
                    hBkgTest.Fill(t)
                else:
                    hBkgTrain.Fill(t)

    inputFile.Close()

In [6]:
# === Apply to both training and testing files ===
fill_histograms("smalltrainData.root", is_test=False)
fill_histograms("smalltestData.root", is_test=True)

Train Tree: sig, Entries: 104918
Train Tree: bkg, Entries: 1565055
Test Tree: sig, Entries: 105370
Test Tree: bkg, Entries: 1563066
                         : Rebuilding Dataset Default


In [9]:
# === Save all histograms to one file ===
histFile = ROOT.TFile("BDTcheck_analysis_full.root", "RECREATE")
for hist in [hSigTrain, hBkgTrain, hSigTest, hBkgTest]:
    hist.Write()
histFile.Close()

In [8]:
#ROC curves can be generated